Ejercicio: ajuste de una curva polinomial

En este ejercicio, veremos un tipo diferente de regresión llamada regresión polinomial. A diferencia de la regresión lineal que modela las relaciones como líneas rectas, la regresión polinomial modela las relaciones como curvas.

Recuerde en nuestro ejercicio anterior cómo la relación entre la temperatura_central y el contenido_proteico_de_la_última_comida no podía explicarse adecuadamente utilizando una línea recta. En este ejercicio, usaremos la regresión polinomial para ajustar una curva a los datos.

visualización de datos
Comencemos este ejercicio cargando y echando un vistazo a nuestros datos.

In [ ]:
import pandas
!pip install statsmodels
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/doggy-illness.csv

#Import the data from the .csv file
dataset = pandas.read_csv('doggy-illness.csv', delimiter="\t")

#Let's have a look at the data
dataset

Regresión lineal simple

Refresquemos rápidamente nuestra memoria realizando la misma regresión lineal simple que hicimos en el ejercicio anterior usando las columnas temperatura y contenido_proteína_de_la_última_comida del conjunto de datos.

In [ ]:
import statsmodels.formula.api as smf
import graphing # custom graphing code. See our GitHub repo for details

# Perform linear regression. This method takes care of
# the entire fitting procedure for us.
simple_formula = "core_temperature ~ protein_content_of_last_meal"
simple_model = smf.ols(formula = simple_formula, data = dataset).fit()

# Show a graph of the result
graphing.scatter_2D(dataset, label_x="protein_content_of_last_meal", 
                             label_y="core_temperature",
                             trendline=lambda x: simple_model.params[1] * x + simple_model.params[0])

Observe cómo la relación entre las dos variables no es realmente lineal. Mirando la gráfica, es bastante claro ver que los puntos tienden más hacia un lado de la línea, especialmente para los valores más altos de temperatura central y contenido_proteico_de_la_última_comida. Una línea recta podría no ser la mejor manera de describir esta relación.

Echemos un vistazo rápido a la puntuación R-Squared del modelo:

In [ ]:
print("R-squared:", simple_model.rsquared)

Esa es una puntuación R-Squared bastante razonable, ¡pero veamos si podemos obtener una aún mejor!

Regresión polinomial simple
Ajustemos una regresión polinomial simple esta vez. De manera similar a una regresión lineal simple, una regresión polinomial simple modela la relación entre una etiqueta y una sola característica. A diferencia de una regresión lineal simple, una regresión polinomial simple puede explicar relaciones que no son simplemente líneas rectas.

En nuestro ejemplo, vamos a utilizar un polinomio de tres parámetros.

In [ ]:
# Perform polynomial regression. This method takes care of
# the entire fitting procedure for us.
polynomial_formula = "core_temperature ~ protein_content_of_last_meal + I(protein_content_of_last_meal**2)"
polynomial_model = smf.ols(formula = polynomial_formula, data = dataset).fit()

# Show a graph of the result
graphing.scatter_2D(dataset, label_x="protein_content_of_last_meal", 
                             label_y="core_temperature",
                             # Our trendline is the equation for the polynomial
                             trendline=lambda x: polynomial_model.params[2] * x**2 + polynomial_model.params[1] * x + polynomial_model.params[0])

Eso ya se ve mucho mejor. Confirmemos echando un vistazo rápido a la puntuación R-Squared:

In [ ]:
print("R-squared:", polynomial_model.rsquared)

Esa es una puntuación R-Squared_ mejor que la obtenida con el modelo anterior, ¡excelente! Ahora podemos decirle con confianza a nuestro veterinario que dé prioridad a los perros que comieron una dieta rica en proteínas la noche anterior.

Tracemos nuestro modelo como un gráfico 3D. vamos a ver
X y X² como dos parámetros separados. Tenga en cuenta que si gira el elemento visual a la derecha, nuestro modelo de regresión sigue siendo un plano. Esta es la razón por la que los modelos polinómicos todavía se consideran modelos lineales.

In [ ]:
import numpy as np
fig = graphing.surface(
    x_values=np.array([min(dataset.protein_content_of_last_meal), max(dataset.protein_content_of_last_meal)]),
    y_values=np.array([min(dataset.protein_content_of_last_meal)**2, max(dataset.protein_content_of_last_meal)**2]),
    calc_z=lambda x,y: polynomial_model.params[0] + (polynomial_model.params[1] * x) + (polynomial_model.params[2] * y),
    axis_title_x="x",
    axis_title_y="x2",
    axis_title_z="Core temperature"
)
# Add our datapoints to it and display
fig.add_scatter3d(x=dataset.protein_content_of_last_meal, y=dataset.protein_content_of_last_meal**2, z=dataset.core_temperature, mode='markers')
fig.show()

Extrapolando

Veamos qué sucede si extraplacamos nuestros datos. Nos gustaría ver si se espera que los perros que comieron alimentos incluso más ricos en proteínas se enfermen aún más.

Comencemos con la regresión lineal. Podemos establecer a qué rango nos gustaría extrapolar nuestros datos usando el argumento x_range en la función de trazado. Extrapolemos sobre el rango [0,100]:

In [ ]:
# Show an extrapolated graph of the linear model
graphing.scatter_2D(dataset, label_x="protein_content_of_last_meal", 
                             label_y="core_temperature",
                             # We extrapolate over the following range
                             x_range = [0,100],
                             trendline=lambda x: simple_model.params[1] * x + simple_model.params[0])

A continuación, extrapolamos la regresión polinomial sobre el mismo rango:

In [ ]:
# Show an extrapolated graph of the polynomial model
graphing.scatter_2D(dataset, label_x="protein_content_of_last_meal", 
                             label_y="core_temperature",
                             # We extrapolate over the following range
                             x_range = [0,100],
                             trendline=lambda x: polynomial_model.params[2] * x**2 + polynomial_model.params[1] * x + polynomial_model.params[0])

¡Estas dos gráficas predicen dos cosas muy diferentes!

La regresión polinómica extrapolada espera que la temperatura central disminuya, mientras que la regresión lineal extrapolada espera que la temperatura central aumente. Una mirada rápida a los gráficos obtenidos en el ejercicio anterior confirma que deberíamos esperar que la temperatura central aumente a medida que aumenta el contenido de proteínas de la última comida, no que disminuya.

En general, no se recomienda extrapolar a partir de una regresión polinomial a menos que tenga una razón a priori para hacerlo (que rara vez ocurre, por lo que es mejor pecar de precavido y nunca extrapolar a partir de regresiones polinómicas). )

Resumen
Cubrimos los siguientes conceptos en este ejercicio:

Cree modelos de regresión lineal simple y regresión polinomial simple.<br>
Compare el rendimiento de ambos modelos al trazarlos y observar los valores R-Squared.<br>
Extrapoló los modelos a un rango más amplio de valores.